# Downloading data notebook

---

## Introduction

This notebook aims to download, process, and store data related to the COVID-19 pandemic.

The main topics covered in this notebook include:

### 1. Environment Setup
   - Required packages and initial setup.

### 2. Data Loading
   - Downloading data
   - Importing the downloaded data.
   - Procedure for downloading the data and saving it locally.

### 3. Data Cleaning and Transformation
   - Handling missing values and formatting columns.
   - Creating new variables or modifying existing ones.

### 4. Saving the Processed Data
   - Saving the transformed data in a suitable format for future use.

### 5. Next Steps
   - Suggestions for future analyses.


### 1. Environment Setup

In [2]:
# Loading packages
library(tidyverse)
library(arrow)
library(tidygeocoder)

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Warning message:
"pacote 'arrow' foi compilado no R versão 4.4.1"

Anexando pacote: 'arrow'


O seguinte objeto é mascarado por 'package:lubridate':

    duration


O seguinte objeto é mascarado por 'package:utils':

    timestamp


Warning message:
"pacote 'tidygeocoder' foi compilado no R versão 4.4.1"


### 2. Data Loading

**About the Dataset:**

This dataset provides daily time series data of COVID-19, capturing the confirmed, recovered, and death cases across various countries worldwide. The dataset includes the following columns:

- **Date**: Ranges from 22/01/2020 to the present day.
- **Country/Region**: The affected countries by COVID-19.
- **Province/State**: Specific states or provinces within the country (if available).
- **Lat**: Latitude of the country or region.
- **Long**: Longitude of the country or region.
- **Confirmed**: The total number of confirmed COVID-19 cases per day.
- **Recovered**: The total number of recovered cases per day.
- **Deaths**: The total number of deaths per day.

**Original Data Source**: [COVID-19 Time Series Data](https://raw.githubusercontent.com/datasets/covid-19/master/data/time-series-19-covid-combined.csv)

In [4]:
# Importing COVID-19 from web
file_names <- "https://raw.githubusercontent.com/datasets/covid-19/master/data/time-series-19-covid-combined.csv"
covid_data <- read_csv(file_names)

Rows: 231744 Columns: 6
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): Country/Region, Province/State
dbl  (3): Confirmed, Recovered, Deaths
date (1): Date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [5]:
# Saving raw data
write_csv_dataset(dataset = covid_data,path = "../data/raw/covid",basename_template = "covid-19-{i}.csv")

In [6]:
dim(covid_data)

[1] 231744      6

In [7]:
head(covid_data)

Date,Country/Region,Province/State,Confirmed,Recovered,Deaths
<date>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
2020-01-22,Afghanistan,NA,0,0,0
2020-01-23,Afghanistan,NA,0,0,0
2020-01-24,Afghanistan,NA,0,0,0
2020-01-25,Afghanistan,NA,0,0,0
2020-01-26,Afghanistan,NA,0,0,0
2020-01-27,Afghanistan,NA,0,0,0


### 3. Data Cleaning and Transformation

In [9]:
# Renaming cols
colnames(covid_data) <- c("date","country_region","province_state","confirmed","recovered","deaths")
head(covid_data)

date,country_region,province_state,confirmed,recovered,deaths
<date>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
2020-01-22,Afghanistan,NA,0,0,0
2020-01-23,Afghanistan,NA,0,0,0
2020-01-24,Afghanistan,NA,0,0,0
2020-01-25,Afghanistan,NA,0,0,0
2020-01-26,Afghanistan,NA,0,0,0
2020-01-27,Afghanistan,NA,0,0,0


In [10]:
# the data points are accumulated by Province/State in some countries
# Lets create a dataset only by countries
covid_by_country_region <- covid_data |> 
    group_by(date,country_region) |>
    summarise(
        confirmed = sum(confirmed),
        recovered = sum(recovered),
        deaths = sum(deaths)
    ) |> 
    ungroup()

head(covid_by_country_region)

`summarise()` has grouped output by 'date'. You can override using the `.groups` argument.


date,country_region,confirmed,recovered,deaths
<date>,<chr>,<dbl>,<dbl>,<dbl>
2020-01-22,Afghanistan,0,0,0
2020-01-22,Albania,0,0,0
2020-01-22,Algeria,0,0,0
2020-01-22,Andorra,0,0,0
2020-01-22,Angola,0,0,0
2020-01-22,Antarctica,0,0,0


In [11]:
# Lets create the daily data points

# First, are the data points completed? 
# Lets check the total for each day and count

length(unique(covid_by_country_region$date)) * length(unique(covid_by_country_region$country_region))
nrow(covid_by_country_region)

# Yes, the data is completed

[1] 161568

[1] 161568

In [12]:
# Lets create the daily data points based on the cummulative confirmed cases
covid_by_country_region <- covid_by_country_region |> 
    arrange(country_region,date) |>
    group_by(country_region) |>
    mutate(daily_confirmed = c(first(confirmed),diff(confirmed))) |> 
    mutate(daily_recovered = c(first(recovered),diff(recovered))) |> 
    mutate(daily_deaths = c(first(deaths),diff(deaths))) |> 
    ungroup()

head(covid_by_country_region)

date,country_region,confirmed,recovered,deaths,daily_confirmed,daily_recovered,daily_deaths
<date>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2020-01-22,Afghanistan,0,0,0,0,0,0
2020-01-23,Afghanistan,0,0,0,0,0,0
2020-01-24,Afghanistan,0,0,0,0,0,0
2020-01-25,Afghanistan,0,0,0,0,0,0
2020-01-26,Afghanistan,0,0,0,0,0,0
2020-01-27,Afghanistan,0,0,0,0,0,0


In [13]:
# Now, Lets create the date variable year, month and day 
covid_by_country_region <- covid_by_country_region |>
    mutate(year  = year(date)) |> 
    mutate(month = month(date)) |> 
    mutate(day   = day(date))  

head(covid_by_country_region)

date,country_region,confirmed,recovered,deaths,daily_confirmed,daily_recovered,daily_deaths,year,month,day
<date>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
2020-01-22,Afghanistan,0,0,0,0,0,0,2020,1,22
2020-01-23,Afghanistan,0,0,0,0,0,0,2020,1,23
2020-01-24,Afghanistan,0,0,0,0,0,0,2020,1,24
2020-01-25,Afghanistan,0,0,0,0,0,0,2020,1,25
2020-01-26,Afghanistan,0,0,0,0,0,0,2020,1,26
2020-01-27,Afghanistan,0,0,0,0,0,0,2020,1,27


In [14]:
# Getting the latitude and longitute based on the countries
lat_longs <- covid_by_country_region  |> 
    select(country_region)  |> 
    unique() |> 
    geocode(country=country_region, method = 'osm', lat = latitude , long = longitude)
head(lat_longs)

Passing 198 addresses to the Nominatim single address geocoder

Query completed in: 201.6 seconds



country_region,latitude,longitude
<chr>,<dbl>,<dbl>
Afghanistan,33.76801,66.238514
Albania,41.00003,19.999962
Algeria,28.00003,2.999983
Andorra,42.54072,1.573203
Angola,-11.87758,17.569124
Antarctica,NA,NA


In [15]:
lat_longs |> 
    filter(is.na(latitude))

country_region,latitude,longitude
<chr>,<dbl>,<dbl>
Antarctica,NA,NA
Diamond Princess,NA,NA
Holy See,NA,NA
MS Zaandam,NA,NA
Summer Olympics 2020,NA,NA
West Bank and Gaza,NA,NA
Winter Olympics 2022,NA,NA


In [16]:
# creating a dataframe to find others addresses
some_addresses <- tibble::tribble(
    ~country_region,                  
    "Antarctica"                                 
    )|> 
    geocode(address = country_region, method = 'osm', lat = latitude , long = longitude)
some_addresses

Passing 1 address to the Nominatim single address geocoder

Query completed in: 1 seconds



country_region,latitude,longitude
<chr>,<dbl>,<dbl>
Antarctica,-72.84387,0


In [17]:
countries_lat_longs <- lat_longs |> 
    filter(!is.na(latitude)) |>
    bind_rows(some_addresses)

In [18]:
covid_by_country_region <- covid_by_country_region |> 
    left_join(countries_lat_longs, by = "country_region")
head(covid_by_country_region)

date,country_region,confirmed,recovered,deaths,daily_confirmed,daily_recovered,daily_deaths,year,month,day,latitude,longitude
<date>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>
2020-01-22,Afghanistan,0,0,0,0,0,0,2020,1,22,33.76801,66.23851
2020-01-23,Afghanistan,0,0,0,0,0,0,2020,1,23,33.76801,66.23851
2020-01-24,Afghanistan,0,0,0,0,0,0,2020,1,24,33.76801,66.23851
2020-01-25,Afghanistan,0,0,0,0,0,0,2020,1,25,33.76801,66.23851
2020-01-26,Afghanistan,0,0,0,0,0,0,2020,1,26,33.76801,66.23851
2020-01-27,Afghanistan,0,0,0,0,0,0,2020,1,27,33.76801,66.23851


In [19]:
# Lets remove the data points which are not a country
# these are the ones 
covid_by_country_region |> 
    filter(is.na(latitude)) |> 
    select(country_region) |> 
    unique()

country_region
<chr>
Diamond Princess
Holy See
MS Zaandam
Summer Olympics 2020
West Bank and Gaza
Winter Olympics 2022


In [20]:
# Creating the country dataset and arranging the column
covid_by_country <- covid_by_country_region |> 
    filter(!is.na(latitude)) |> 
    select(date, country_region, confirmed, daily_confirmed, recovered, daily_recovered, deaths, daily_deaths, year, month, day)
head(covid_by_country)

date,country_region,confirmed,daily_confirmed,recovered,daily_recovered,deaths,daily_deaths,year,month,day
<date>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
2020-01-22,Afghanistan,0,0,0,0,0,0,2020,1,22
2020-01-23,Afghanistan,0,0,0,0,0,0,2020,1,23
2020-01-24,Afghanistan,0,0,0,0,0,0,2020,1,24
2020-01-25,Afghanistan,0,0,0,0,0,0,2020,1,25
2020-01-26,Afghanistan,0,0,0,0,0,0,2020,1,26
2020-01-27,Afghanistan,0,0,0,0,0,0,2020,1,27


### 4. Saving the Processed Data

In [22]:
# Sometimes we have large dataset, lets use our example to use arrow package to save the data in partitions in a parquet format
# Let's partition the data using the variables year, month, and day

write_dataset(
    covid_by_country, 
    path = "../data/enriched/covid", 
    format = "parquet", 
    partitioning = c("year","month","day")
)

### 5. Next Steps

Here are some potential analyses we could conduct with our data:

1. **Detailed Descriptive Analysis**
   - Conduct a more in-depth descriptive analysis of the COVID-19 data. This can include calculating key metrics such as daily growth rates, moving averages of new cases, and death-to-recovery ratios. Additional visualizations like histograms, box plots, and heatmaps can further explore trends and distributions across different time periods and regions.

3. **Time Series Analysis**
   - Explore time series models to analyze trends in COVID-19 case numbers, recoveries, and fatalities over time. Techniques such as ARIMA or Prophet could be used to model and forecast future case counts based on historical data.

4. **Geospatial Analysis**
   - Perform a geospatial analysis to map the distribution of COVID-19 cases by region or country. Using maps and clustering techniques, it’s possible to identify spatial patterns or regional hotspots, providing insights into geographic spread.